In [6]:
%pylab qt

import os
import itertools
import gzip
import cPickle as pickle
import cv2
import numpy as np
from math import radians, degrees
from glob import glob
from scipy.signal import argrelextrema
from sklearn.preprocessing import normalize
from skimage.transform import estimate_transform
from scipy.interpolate import splprep, splev

import helpers.classes as ch
import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
reload(ch)
reload(fh)
reload(dh)
reload(gh)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_mesh_2/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_filtered_and_registered_outline/'
NUM_SPLINE_POINTS = 100

def extract_landmark_points(outline_points, outline_edges):
    step_size = 1
    landmark_definitions = [
        {
            'a_min': 30,
            'a_max': 90,
            'method': 'max'
        },
        {
            'a_min': 80,
            'a_max': 100,
            'method': 'min'
        },
        {
            'a_min': 90,
            'a_max': 150,
            'method': 'max'
        },
        {
            'a_min': 160,
            'a_max': 200,
            'method': 'max'
        },
        {
            'a_min': 210,
            'a_max': 270,
            'method': 'max'
        },
        {
            'a_min': 260,
            'a_max': 280,
            'method': 'min'
        },
        {
            'a_min': 270,
            'a_max': 330,
            'method': 'max'
        }
    ]
    landmarks = []
    rho = 2
    centroid = mean(outline_points, axis=0)
    
    for definition in landmark_definitions:
        intersects_for_angles = []
        method = np.argmax if definition['method'] == 'max' else np.argmin
        
        for angle in range(definition['a_min'], definition['a_max']):
            startpoint = centroid
            endpoint = gh.pol2cart(rho, radians(angle)) + centroid
            
            intersect = None
            for edge in outline_edges:
                p1 = outline_points[edge[0], :]
                p2 = outline_points[edge[1], :]
                
                intersect = gh.seg_intersect(p1, p2, startpoint, endpoint)
                if intersect is not None:
                    break
            if intersect is None:
                raise Exception('No Intersect.')
            else:
                intersects_for_angles.append(intersect)
        
        intersects_for_angles = np.array(intersects_for_angles)
        distances = np.linalg.norm(intersects_for_angles - np.tile(centroid, (intersects_for_angles.shape[0], 1)), axis=1)
        
        landmarks.append(intersects_for_angles[method(distances), :])
        
    return np.array(landmarks)

def register_outline_with_icp(reference, reference_edges, points, edges, no_iterations = 1):
    landmarks_reference = extract_landmark_points(reference, reference_edges)

    for i in range(no_iterations):
        landmarks = extract_landmark_points(points, edges)
        #Compute the transformation between the current source
        #and destination cloudpoint
        #tform = estimate_transform('affine', landmarks, landmarks_reference)
        #points = tform(points)
        
        R, t = gh.estimate_rigid_transform(landmarks, landmarks_reference)
        R = np.array(R)
        t = np.array(t)
        #Transform the previous source and update the
        #current source cloudpoint
        points = (np.dot(R, points.transpose())).transpose() + np.tile(t, (points.shape[0], 1))
    
    return points

def extract_spline(points):
    y = points[:,0].flatten()
    x = points[:,1].flatten()
    
    tck, u = splprep([y, x], s=0)
    coords = splev(np.linspace(0, 1, NUM_SPLINE_POINTS), tck)
    
    spline_points = np.zeros((NUM_SPLINE_POINTS, 2))
    spline_points[:, 0] = coords[0]
    spline_points[:, 1] = coords[1]
    
    return spline_points

def do_registration(loaded):
    transposed_and_scaled = []
    registered = []
    
    for outline in loaded:
        points = outline['points']
        
        spl_points = extract_spline(points)
        centroid = mean(spl_points, axis=0)
        points = points - np.tile(centroid, (len(points), 1))
        spl_points = spl_points - np.tile(centroid, (len(spl_points), 1))
        
        #scale = sqrt(np.sum(np.power(spl_points[:, 0], 2)) / len(spl_points))
        scale = sqrt(np.sum(np.power(spl_points, 2)) / len(spl_points))
        points = np.divide(points, scale)
        points, edges = gh.normalize_outline(points, outline['edges'])
        
        transposed_and_scaled.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'class': outline['class'],
            'class_label': outline['class_label'],
            'points': points,
            'edges': edges
        })
    
    reference = max(transposed_and_scaled, key=lambda o: o['points'].shape[0])
    for outline in transposed_and_scaled:
        points = outline['points']
        edges = outline['edges']
        
        if not np.array_equal(points, reference):
            points = register_outline_with_icp(reference['points'], reference['edges'], points, outline['edges'])
            points, edges = gh.normalize_outline(points, edges)
        registered.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'class': outline['class'],
            'class_label': outline['class_label'],
            'points': points,
            'edges': edges
        })
    
    return registered

Populating the interactive namespace from numpy and matplotlib
Current base path: /home/stefan/Dropbox/Masterarbeit


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
filenames = glob(DATA_PATH + '*.pkl')
loaded = []
for filename in filenames:
    with gzip.GzipFile(filename, 'rb') as f:
        content = pickle.load(f)
        
    if 'done' in content and content['done']:
        basename = os.path.basename(filename)
        points = content['points']
        triangles = content['simplices']

        label = ''.join([i if ord(i) < 128 else ' ' for i in basename])

        cls = ch.get_class(basename)

        outline_points, outline_edges = gh.extract_outline(points, triangles)

        loaded.append({
            'label': label,
            'filename': basename,
            'class': cls[0],
            'class_label': cls[1],
            'points': outline_points,
            'edges': outline_edges
        })

In [3]:
filtered = ch.filter_by_classes(loaded, [ 2 ])

In [4]:
dh.outlines(filtered, color_by_class=True)

In [7]:
registered = do_registration(filtered)

In [8]:
dh.outlines(registered, color_by_class=True)

In [20]:
outline = registered[5]
landmarks = extract_landmark_points(outline['points'], outline['edges'])
window = dh.outline(outline, show_direction=True)
pointActor = dh.get_points_actor(landmarks)
window.ren.AddActor(pointActor)
window.vtkWidget.GetRenderWindow().Render()

In [28]:
for outline in registered:
    basename = os.path.basename(outline['filename'])
    destination = os.path.join(TO_PATH, basename)
    
    np.savez(destination, points=outline['points'], edges=outline['edges'])